In [1]:
import pandas as pd


In [25]:
import torch
import numpy as np


In [2]:
train = pd.read_json("dataset/training/EXIST2023_training.json", orient="index")


In [3]:
train = train.sample(frac=1).reset_index(drop=True)


In [28]:
train


,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split
0,102440,es,"@saboramieI No existe la misandria, tho. No te...",6,"[Annotator_277, Annotator_278, Annotator_279, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, NO, NO, YES, NO]","[REPORTED, DIRECT, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [IDEOLOGICAL-INEQUA...",TRAIN_ES
1,103328,es,@morbogalicia no se si me gusta mas el sujetad...,6,"[Annotator_301, Annotator_302, Annotator_303, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, YES, YES, YES, YES, NO]","[-, DIRECT, DIRECT, DIRECT, DIRECT, -]","[[-], [OBJECTIFICATION], [SEXUAL-VIOLENCE], [O...",TRAIN_ES
2,103349,es,Nadie te va a tratar tan bien como un hombre q...,6,"[Annotator_73, Annotator_74, Annotator_75, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, NO, YES, YES, YES, NO]","[DIRECT, -, REPORTED, DIRECT, DIRECT, -]","[[OBJECTIFICATION], [-], [SEXUAL-VIOLENCE], [S...",TRAIN_ES
3,102012,es,@LaComadritaOf2 @edwinlunat Esa vieja cae gord...,6,"[Annotator_265, Annotator_266, Annotator_267, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, DIRECT, DIRECT, ...","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",TRAIN_ES
4,102018,es,@SinEmbargoMX La grande por ahí anda un presum...,6,"[Annotator_97, Annotator_98, Annotator_99, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, DIRECT, DIRECT]","[[-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-], [ST...",TRAIN_ES
...,...,...,...,...,...,...,...,...,...,...,...
6915,200303,en,Just ate thee hot ass chips I can’t feel my to...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, DIRECT, -]","[[-], [-], [-], [-], [OBJECTIFICATION], [-]]",TRAIN_EN
6916,200395,en,@blautags @1kralim @Ertyks @IS0FX @ADMMA41 @er...,6,"[Annotator_578, Annotator_579, Annotator_580, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_EN
6917,103542,es,"De esto sí vale la pena asustarse, y no de eso...",6,"[Annotator_157, Annotator_158, Annotator_159, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, YES, NO, NO, NO]","[-, -, JUDGEMENTAL, -, -, -]","[[-], [-], [STEREOTYPING-DOMINANCE], [-], [-],...",TRAIN_ES
6918,101069,es,Primer #EnIgualdad del año en @BrujulaOndaCero...,6,"[Annotator_223, Annotator_224, Annotator_225, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_ES


In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base", num_labels=2
)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [5]:
training_columns = train.columns


In [6]:
training_columns


Index(['id_EXIST', 'lang', 'tweet', 'number_annotators', 'annotators',
       'gender_annotators', 'age_annotators', 'labels_task1', 'labels_task2',
       'labels_task3', 'split'],
      dtype='object')

In [7]:
text = train["tweet"]


In [12]:
encoded_text = [tokenizer(sent, return_tensors="pt") for sent in text]


In [15]:
encoded_text


[{'input_ids': tensor([[     0,   1374,    433,  27651,   6224,    568,    438,  13835,     21,
            1239, 150213,     11,      4,  14461,      5,    438,  30412,   4105,
               4,     78,  66004,      5,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[    0,  1374,  4514, 83067, 28620,    11,   110,    40,    78,   163,
          16057,  1163,    88, 24693,  9601,    36,   576,   117,   150,  1073,
           1788,   198,   472,   110,    40,   173,    78,    22,  1076,  1806,
            435, 14692, 30412,    41,   493,  2512,  1163,     8, 11099,   472,
          78261,     7,     7,     7,  1564,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[     0,    353,   7814,    120,    307,     10,  48731,   2197,   1806,
     

In [20]:
output = model(**encoded_text[0])


In [24]:
output.logits


tensor([[[62.5218, -0.1210, 42.0036,  ..., 25.1463, 16.4296, 21.4097],
         [22.1814, -0.7230, 46.9252,  ..., 30.0188, 15.6469, 25.6064],
         [17.4978, -0.9722, 44.9620,  ..., 40.8827, 18.1030, 25.5776],
         ...,
         [18.1637, -0.9583, 47.9548,  ..., 29.1164, 17.0192, 25.9155],
         [30.6634, -1.1966, 65.2536,  ..., 45.4879, 19.6610, 34.5213],
         [34.1157, -0.4043, 54.2395,  ..., 32.2780, 18.5180, 26.0352]]],
       grad_fn=<ViewBackward0>)